In [3]:
from tensorflow.keras.datasets import mnist
from sklearn.linear_model import LogisticRegression
import joblib

print("Loading MNIST from Keras...")
(X_train, y_train), _ = mnist.load_data()

# Flatten 28x28 -> 784
X_train = X_train.reshape((X_train.shape[0], -1)).astype("float32") / 255.0

print("Training Logistic Regression...")
model = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
model.fit(X_train, y_train)

print("Saving model...")
joblib.dump(model, "mnist_logreg.pkl")
print("Model saved as mnist_logreg.pkl")

Loading MNIST from Keras...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training Logistic Regression...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Saving model...
Model saved as mnist_logreg.pkl


In [5]:
import numpy as np
import gzip
from sklearn.linear_model import LogisticRegression
import joblib

def load_images(filename):
    with gzip.open(filename, 'rb') as f:
        f.read(16)  # skip magic number & dimensions
        data = np.frombuffer(f.read(), dtype=np.uint8)
        data = data.reshape(-1, 28*28) / 255.0
    return data

def load_labels(filename):
    with gzip.open(filename, 'rb') as f:
        f.read(8)  # skip magic number & items
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

print("Loading MNIST from local .gz files...")
X_train = load_images("train-images-idx3-ubyte.gz")
y_train = load_labels("train-labels-idx1-ubyte.gz")

print("Training Logistic Regression...")
model = LogisticRegression(
    max_iter=1000,
    solver='lbfgs',
    multi_class='multinomial',
    verbose=1
)
model.fit(X_train, y_train)

print("Saving model...")
joblib.dump(model, "mnist_2.pkl")
print("✅ Model saved as mnist_2.pkl")

Loading MNIST from local .gz files...
Training Logistic Regression...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Saving model...
✅ Model saved as mnist_2.pkl


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished
